In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [6]:
number_of_files = len(os.listdir('PL Data'))
# print(number_of_files)

df_list = []

for i in range(number_of_files):
    temp = r'C:\Users\RSP-Mini\Onedrive\OneDrive JupyterNotebook\3172 JupyterNotebook\PL Data\\' + str(os.listdir('PL Data')[i])
    df_list.append(pd.read_csv(temp, skiprows=0, sep="\t"))
    # Un-hastag below to plot graphs
    #df_list[i].plot(df_list[i].columns[0], subplots=True, figsize=(15,5), title=f'Target Wire #{i+1} after SEM', 
    #                xlabel='Wavelength (nm)', ylabel='Intensity (Count)', ylim=(0,10000), grid=0.5)
                    # Note the y-limit of 10,000

####################################################    
# Print number of data points = 7449 #
####################################################
for i in range(0, len(df_list)):
    print(len((df_list[i].Wavelength)))

7449
7449
7449
7449
7449
7449
7449
7449
7449
7449
7449
7449


In [7]:
df_list[0]

,Wavelength,Intensity
0,700.006,4821.7100
1,699.962,4771.6000
2,699.918,4865.5100
3,699.874,4900.9500
4,699.830,4872.3800
...,...,...
7444,350.148,53.4122
7445,350.099,50.2561
7446,350.051,60.2903
7447,350.002,55.8784


In [8]:
###########################
# Code to identify spikes 
###########################
# We will compare the nth value with the (n+1)th value, if (n+1)/n > threshold then (n+1) is identified as a spike.
def spikes_identify(dataset):
    threshold = 1.2
    temp_list = []
    for i in range(6600,7100):
        temp_list.append(dataset.loc[i, 'Intensity'])
    temp = max(temp_list)
    if temp > 5000:
        threshold_high = temp
        print(f'threshold_high is {threshold_high}')
    else:
        threshold_high=100000
        print(f'The exciton peak is too small (I={temp}) for First Filter to work.')

    spikes = []
    spike_count = 0

    initial_y_value = dataset.loc[0, 'Intensity']
    # print(initial_y_value)
    prev_y_value = initial_y_value

    for i in range(1, len(dataset.Intensity)-1):
        current_y_value = dataset.loc[i, 'Intensity']

        ## First filter: check if the absolute value for each Intensity is above the max of the exciton peak AND if the adjacent
        ## intensity value is also above (expected) or under (unexpected, truly anomalous peak then)

        if current_y_value > threshold_high and prev_y_value < threshold_high: 
            x_coord_of_spike = dataset.Wavelength[i]
            print(f'i: {i} \t x_value of spike: {x_coord_of_spike} \t current_y_value = {current_y_value} \t prev (non-peaked) y_value = {prev_y_value}')
            spikes.append(i)
            spike_count += 1

        ## Second filter: check if the absolute value for each Intensity is more than (threshold) times of the next value

        elif current_y_value > prev_y_value:
            if current_y_value/prev_y_value > threshold:
                x_coord_of_spike = dataset.Wavelength[i]
                print(f'i: {i} \t x_value of spike: {x_coord_of_spike} \t current_y_value = {current_y_value} \t prev (non-peaked) y_value = {prev_y_value}')
                spikes.append(i)
                spike_count += 1
                # prev_y_value = prev_y_value since the current value is anomalous. Do not un-hashtag this.
            else:
                prev_y_value = dataset.loc[i+1, 'Intensity']

        ## Third filter: check if the absolute value for each Intensity is less than (threshold) times of the next value
        ## (this is for when current_y_value is a peaked value)

        elif current_y_value < prev_y_value:
            if current_y_value/prev_y_value < 1/threshold:
                x_coord_of_spike = dataset.Wavelength[i-1]
                print(f'i: {i-1} \t x_value of spike: {x_coord_of_spike} \t current_y_value = {prev_y_value} \t next_y_value = {current_y_value}')
                spikes.append(i-1)
                spike_count += 1

                prev_y_value = current_y_value

    print(f'Spikes found: {spike_count}')
    return spikes

In [9]:
#########################
# Code to remove spikes #
#########################
def spikes_remove(dataset, list_of_spikes):
    temp_dataset = dataset.copy()
    for i in list_of_spikes:
        temp_dataset.Intensity[i] = np.nan
    return temp_dataset

In [10]:
##############################
# Code to interpolate spikes #
##############################
def spikes_interpolate(dataset):
    return dataset.interpolate()

In [11]:
###################################################################
# Code to identify, remove and interpolate spikes of all datasets #
###################################################################
def spikes_cleaner(dataset):
    return spikes_interpolate(spikes_remove(dataset, spikes_identify(dataset)))